<a href="https://colab.research.google.com/github/redman157/HocML/blob/master/HocPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 16, 5)
        self.conv2 = nn.Conv2d(16, 64, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(64 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
input = torch.randn(1, 1, 32, 32)
print(input,"\n",input.shape)
out = net(input)
print(out)

tensor([[[[-1.5362,  0.8645, -0.7842,  ..., -0.2506,  0.3524, -0.8021],
          [ 0.2462,  0.3740, -0.2126,  ...,  0.5569, -0.8287, -1.0154],
          [-0.4887, -1.1374,  0.2350,  ..., -0.3305, -0.2740,  0.5572],
          ...,
          [-1.4702, -0.7062, -0.0509,  ...,  0.6078, -0.3471, -1.1502],
          [ 0.6462, -1.5002,  0.0776,  ..., -0.6081,  0.8383,  0.4824],
          [-2.3282, -0.3576,  0.5836,  ..., -0.3348,  0.9025,  0.3260]]]]) 
 torch.Size([1, 1, 32, 32])
tensor([[-0.0055,  0.0225, -0.0116,  0.0068,  0.0463,  0.1020,  0.1336,  0.0064,
          0.0841,  0.0470]], grad_fn=<AddmmBackward>)


In [0]:
net.zero_grad()
out.backward(torch.randn(1,10))


In [6]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)

criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.5354, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [8]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0080,  0.0028,  0.0175, -0.0004,  0.0019,  0.0207,  0.0068, -0.0129,
         0.0142,  0.0077, -0.0004, -0.0147,  0.0095, -0.0045,  0.0024,  0.0024])


In [0]:
optimizer = optim.Adam(net.parameters(), lr = 0.001)
optimizer.zero_grad()
output = net(input)
criterion = nn.MSELoss()
loss = criterion(output, target)
loss.backward()
optimizer.step()